In [1]:
import re

In [2]:
s=re.match(r'\d{3}\-\d{8}','123-45123457')

In [3]:
re.match(r'^\d{3}\-\d{3,8}$', '010-12345')

<re.Match object; span=(0, 9), match='010-12345'>

In [4]:
q=re.match(r'^\d{3}\-\d{3,8}$', '010 12345')

In [5]:
if re.match(r'\d{3}\-\d{8}','123-45123457'):
    print('suceess')
else:
    print('faile')

suceess


In [6]:
s.group(0)

'123-45123457'

In [7]:
f=open('And.hdl','r')

In [8]:
q=f.read()

In [9]:
q

'// This file is part of www.nand2tetris.org\n// and the book "The Elements of Computing Systems"\n// by Nisan and Schocken, MIT Press.\n// File name: projects/01/And.hdl\n\n/**\n * And gate: \n * out = 1 if (a == 1 and b == 1)\n *       0 otherwise\n */\n\nCHIP And {\n    IN  a, b;\n    OUT out;\n    PARTS:\n    Nand(a=a,b=b,out=out1);\n    Not(in=out1,out=out);\n    // Put your code here:\n}\n'

In [75]:
q1=q.split('\n')

In [77]:
q1[0]

'// This file is part of www.nand2tetris.org'

In [28]:
q1[1].sub(' ')

AttributeError: 'str' object has no attribute 'sub'

In [36]:
_comment = re.compile('//.*$')

In [40]:
q2=_comment.sub('',q1)

TypeError: expected string or bytes-like object

In [2]:
class assumbler():
    def __init__(self):
        self.bultinaddress= {'SP':0, 'LCL':1, 'ARG':2, 'THIS':3, 'THAT':4, 
                        'R0':0, 'R1':1, 'R2':2, 'R3':3, 'R4':4, 'R5':5,
                        'R6':6, 'R7':7, 'R8':8, 'R9':9, 'R10':10, 'R11':11,
                        'R12':12, 'R13':13, 'R14':14, 'R15':15, 
                        'SCREEN':16384, 'KBD':24576}
        self.readyaddress=16
    def handle_label(self,u):
        linenumber=0
        for i in u:
            if i[0]=='(' and i[-1] ==')':
                o=i[1:-1]
                print(linenumber)
                if o in self.bultinaddress.keys():
                    raise Exception("Assembler: attempted to add already-existing label: " + line)
                self.bultinaddress[o]=linenumber
                continue
            linenumber+=1    
    def denum(self,q1):
        if q1<0 or q1>2**15:
            raise Exception("Assembler: out-of-range assignment to A!:  " + q1)
        q=bin(q1)
        q=q[2:]
        numdigits = min(15, len(q))
        numzeros = 16-numdigits
        bincmd='0'*numzeros + q[-numdigits:]
        return bincmd
    
    def a_expression(self,s):
        if s[1:].isdigit():
            w=int(s[1:])
            addr=self.denum(w)
        elif self.islegal(s[1:]):
            e=s[1:]
            
            if e not in self.bultinaddress.keys():
                self.bultinaddress[e]=self.readyaddress
                self.readyaddress+=1
            addr=self.denum(int(self.bultinaddress[e]))
        else:
            raise Exception("Assembler: illegal variable name:  " + line)
        return addr   
    
    def c_cut(self,y):
        m=-1
        n=len(y)
        dest=None
        comp=None
        jump=None
        if '=' in y:
            m=y.index('=')
            dest=y[:m]
        if ';' in y:
            n=y.index(';')
            jump=y[n+1:]
        comp=y[m+1:n]
       # print(m,n)
        return dest,comp,jump
    def handle_dest(self, dest):
        # TODO: Ignores poorly-formed dest commands
        if dest is None:
            destcmd = '000'
        else:
            destcmd = ''
            if 'A' in dest:
                destcmd = destcmd + '1'
            else:
                destcmd = destcmd + '0'
            if 'D' in dest:
                destcmd = destcmd + '1'
            else:
                destcmd = destcmd + '0'
            if 'M' in dest:
                destcmd = destcmd + '1'
            else:
                destcmd = destcmd + '0'
        return destcmd

    # parses jump command - raises Exception if bad command
    def handle_jump(self, jump):
        if jump is None:
            jumpcmd = '000'
        elif jump == 'JGT':
            jumpcmd = '001'
        elif jump == 'JEQ':
            jumpcmd = '010'
        elif jump == 'JGE':
            jumpcmd = '011'
        elif jump == 'JLT':
            jumpcmd = '100'
        elif jump == 'JNE':
            jumpcmd = '101'
        elif jump == 'JLE':
            jumpcmd = '110'
        elif jump == 'JMP':
            jumpcmd = '111'
        else:
            raise Exception("Assembler: Illegal jump command: " + jump)
        return jumpcmd
    
    # there are a limited number of allowed calculations ...
    # so, effectively do a switch on them. This feels dirty.
    def handle_comp(self, calc):
        if calc == '0':
            calccmd = '0101010'
        elif calc == '1':
            calccmd = '0111111'
        elif calc == '-1':
            calccmd = '0111010'
        elif calc == 'D':
            calccmd = '0001100'
        elif calc == 'A':
            calccmd = '0110000'
        elif calc == '!D':
            calccmd = '0001101'
        elif calc == '!A':
            calccmd = '0110001'
        elif calc == '-D':
            calccmd = '0001111'
        elif calc == '-A':
            calccmd = '0110011'
        elif calc == 'D+1' or calc == '1+D':
            calccmd = '0011111'
        elif calc == 'A+1' or calc == '1+A':
            calccmd = '0110111'
        elif calc == 'D-1':
            calccmd = '0001110'
        elif calc == 'A-1':
            calccmd = '0110010'
        elif calc == 'D+A' or calc == 'A+D':
            calccmd = '0000010'
        elif calc == 'D-A':
            calccmd = '0010011'
        elif calc == 'A-D':
            calccmd = '0000111'
        elif calc == 'D&A' or calc == 'A&D':
            calccmd = '0000000'
        elif calc == 'D|A' or calc == 'A|D':
            calccmd = '0010101'
        elif calc == 'M':
            calccmd = '1110000'
        elif calc == '!M':
            calccmd = '1110001'
        elif calc == '-M':
            calccmd = '1110011'
        elif calc == 'M+1' or calc == '1+M':
            calccmd = '1110111'
        elif calc == 'M-1':
            calccmd = '1110010'
        elif calc == 'D+M' or calc == 'M+D':
            calccmd = '1000010'
        elif calc == 'D-M':
            calccmd = '1010011'
        elif calc == 'M-D':
            calccmd = '1000111'
        elif calc == 'D&M' or calc == 'M&D':
            calccmd = '1000000'
        elif calc == 'D|M' or calc == 'M|D':
            calccmd = '1010101'
        else:
            raise Exception("Assembler: illegal calculation: " + calc)
        return calccmd
    def islegal(self, str):
        if str[0].isdigit():
            return False
        for sym in ['-', '*', '+', '/', '&', '|', '!']:
            if sym in str:
                return False
        else:
            return True
          
    def c_expression(self,t):
        dest,comp,jump=self.c_cut(t)
      #  print(comp,type(comp),len(comp))
        jumpcode=self.handle_jump(jump)
        compcode=self.handle_comp(comp)
        destcode=self.handle_dest(dest)
        return '111'+compcode+destcode+jumpcode
def clean(q):
    out=[]
    for i in q:
        if '/' in i:
            m=i.index('/')
            i=i[:m]
        if i=='':
            continue
        i=i.replace(" ", "")
        out.append(i)
    return out  

In [3]:
myassembler.bultinaddress

NameError: name 'myassembler' is not defined

In [9]:
def main():
#    a=sys.argv
#    if len(a)!=3:
#        raise Exception("two input parameter")
#    infile=sys.argv[1]
#    outfile=sys.argv[2]
    infile='Add.asm'
    outfile='Add.hack'
    myassembler = assumbler()
    f=open(infile,'r')
    q=f.read()
    q=q.split('\n')
    q=clean(q)
    #print(q)
    myassembler.handle_label(q)
    print(myassembler.bultinaddress)
    f1=open(outfile,'w')
    for i in q:
        if i[0]=='(' and i[-1] ==')':
            continue
        if i[0]=='@':
            p=myassembler.a_expression(i)
        else:     
            p=myassembler.c_expression(i)
        f1.write(p+'\n')
    f.close()
    f1.close()

In [10]:
main()

{'SP': 0, 'LCL': 1, 'ARG': 2, 'THIS': 3, 'THAT': 4, 'R0': 0, 'R1': 1, 'R2': 2, 'R3': 3, 'R4': 4, 'R5': 5, 'R6': 6, 'R7': 7, 'R8': 8, 'R9': 9, 'R10': 10, 'R11': 11, 'R12': 12, 'R13': 13, 'R14': 14, 'R15': 15, 'SCREEN': 16384, 'KBD': 24576}


In [299]:
qq=bin(10)

In [204]:
a=['dadada','fqqq','ngcf']

In [206]:
a

['dadada', 'fqqq', 'ngcf']

In [207]:
del a[0]

In [209]:
a.remove('fqqq')

In [210]:
a

['ngcf']

In [268]:
infile='Fill.asm'
outfile='Fill.hack'
myassembler = assumbler()
f=open(infile,'r')
q=f.read()
q=q.split('\n')
q=clean(q)
#print(q)
myassembler.handle_label(q)
#f1=open(outfile,'w')
#for i in q:
#    if i[0]=='(' and i[-1] ==')':
#        continue
#    if i[0]=='@':
#        p=myassembler.a_expression(i)
#    else:     
#        p=myassembler.c_expression(i)
#    f1.write(p+'\n')
#f.close()
#f1.close()